<a href="https://colab.research.google.com/github/dk-wei/aws-ml/blob/main/AWS_Lambda_and_Amazon_API_Gateway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: [Call an Amazon SageMaker model endpoint using Amazon API Gateway and AWS Lambda](https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/)

After we deployed our ML model on in production by creating a SageMaker model endpoint, we need to build a architecture to call our model and return prediction output. The regular practice is to invoke the SageMaker endpoint using API Gateway and Lambda function:

1. Starting from the client side, a client script calls an Amazon API Gateway API action and passes parameter values (API Gateway is a layer that provides the API to the client).
2. The `Lambda function` parses the value and sends it to the SageMaker model endpoint. 
3. The model performs the prediction and returns the predicted value to Lambda. The Lambda function parses the returned value and sends it back to API Gateway. API Gateway responds to the client with that value.

![](https://mindsdb.com/wp-content/uploads/2020/11/5e5585613c280e8d01c846ef_L3UzaupMh6xMes5rk9GZMBg7TXNPZvPnXrQc58knp-6JRZhjdw9UiBuDDDEDSNQsKhol3Y5NLyTisl-CrWre3AmQiGuO5UYRoDd6Gggzib6rLb8t9Ymi0x4EuVLU8XytH2zvBtEj.png)





Complete steps:
1. Train your model and Create a SageMaker model endpoint by any means.
2. Create a Lambda function that calls the SageMaker runtime invoke_endpoint
3. Create a REST API: Integration request setup
4. Test data (with postman)


Sample Lambda function code:

In [ ]:
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']  

# use boto3 to get sagemaker runtime
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    print(payload)
    
    # this will allow us pass the input data and invoke sagemaker runtime
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/csv',
                                       Body=payload)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    pred = int(result['predictions'][0]['score'])
    predicted_label = 'M' if pred == 1 else 'B'
    
    return predicted_label

Sample Amazon Gateway config:

Once we call the api, the `Lambda` function will triggered and invoke the endpoint to do the prediction, then return output

![](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/03/31/10-Actions-Method-execution.jpg)